In [21]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv("dataset/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
classifier=KNeighborsClassifier(n_neighbors=5,algorithm='auto')
classifier.fit(X_train,y_train)

KNeighborsClassifier()

In [11]:
y_pred=classifier.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [13]:
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[130  38]
 [ 38  48]]
0.7007874015748031
              precision    recall  f1-score   support

           0       0.77      0.77      0.77       168
           1       0.56      0.56      0.56        86

    accuracy                           0.70       254
   macro avg       0.67      0.67      0.67       254
weighted avg       0.70      0.70      0.70       254



The accuracy is not high enough

In [14]:
# Count the number of zeros in each column
zero_count = (df == 0).sum()

print("Number of zeros in each column:")
print(zero_count)

Number of zeros in each column:
Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64


In [15]:
# Before handling zeros
print("Zeros in each column before handling:")
print((df == 0).sum())

# Replace zeros with NaN in specific columns
columns_to_fix = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[columns_to_fix] = df[columns_to_fix].replace(0, np.nan)

# Impute missing values with median
for column in columns_to_fix:
    df[column].fillna(df[column].median(), inplace=True)

# After handling zeros
print("Zeros in each column after handling:")
print((df == 0).sum())

Zeros in each column before handling:
Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64
Zeros in each column after handling:
Pregnancies                 111
Glucose                       0
BloodPressure                 0
SkinThickness                 0
Insulin                       0
BMI                           0
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64


C:\Users\nage\AppData\Local\Temp\ipykernel_7752\1884891265.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].median(), inplace=True)
C:\Users\nage\AppData\Local\Temp\ipykernel_7752\1884891265.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [31]:
def detect_outliers(df, features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile Q1
        Q1 = np.percentile(df[c], 25)
        # 3st quartile Q3
        Q3 = np.percentile(df[c], 75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indices
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indices
        outlier_indices.extend(outlier_list_col)
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [32]:
detect_outliers(df, ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

[125, 177, 662, 39, 120, 370, 445]

In [33]:
# drop outliers
df = df.drop(detect_outliers(df, ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']), axis = 0).reset_index(drop= True)

In [34]:
df.shape

(761, 9)

In [35]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [38]:
classifier=KNeighborsClassifier(n_neighbors=5,algorithm='auto')
classifier.fit(X_train,y_train)

KNeighborsClassifier()

In [39]:
y_pred=classifier.predict(X_test)

In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [41]:
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

[[137  30]
 [ 37  48]]
0.7341269841269841
              precision    recall  f1-score   support

           0       0.79      0.82      0.80       167
           1       0.62      0.56      0.59        85

    accuracy                           0.73       252
   macro avg       0.70      0.69      0.70       252
weighted avg       0.73      0.73      0.73       252



No much improve in accuracy lets try grid search with differnt n neighbors

In [42]:
from sklearn.model_selection import GridSearchCV

In [43]:
# defining parameter range
param_grid = {'n_neighbors' : [1,2,3,4,5,6,7,8,9,10]}

In [44]:
grid=GridSearchCV(KNeighborsClassifier(),param_grid=param_grid,refit=True,cv=5,verbose=3)

In [45]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....................n_neighbors=1;, score=0.686 total time=   0.0s
[CV 2/5] END .....................n_neighbors=1;, score=0.657 total time=   0.0s
[CV 3/5] END .....................n_neighbors=1;, score=0.676 total time=   0.0s
[CV 4/5] END .....................n_neighbors=1;, score=0.667 total time=   0.0s
[CV 5/5] END .....................n_neighbors=1;, score=0.703 total time=   0.0s
[CV 1/5] END .....................n_neighbors=2;, score=0.676 total time=   0.0s
[CV 2/5] END .....................n_neighbors=2;, score=0.618 total time=   0.0s
[CV 3/5] END .....................n_neighbors=2;, score=0.667 total time=   0.0s
[CV 4/5] END .....................n_neighbors=2;, score=0.716 total time=   0.0s
[CV 5/5] END .....................n_neighbors=2;, score=0.733 total time=   0.0s
[CV 1/5] END .....................n_neighbors=3;, score=0.676 total time=   0.0s
[CV 2/5] END .....................n_neighbors=3;

C:\Users\nage\develop\K Nearest Neighbor\env\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             verbose=3)

In [46]:
grid.best_params_

{'n_neighbors': 8}

In [47]:
y_pred2=grid.predict(X_test)
print(classification_report(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       174
           1       0.65      0.55      0.60        78

    accuracy                           0.77       252
   macro avg       0.73      0.71      0.72       252
weighted avg       0.76      0.77      0.76       252

[[151  23]
 [ 35  43]]


In [48]:
import pickle

In [50]:
# Create a pickle file and write the data
with open('diabetes_model.pkl', 'wb') as file:
    pickle.dump(grid, file)